In [1]:
import annoy
import sys
import pandas as pd
import os
import numpy as np
import sre_parse
import implicit
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
import sys
sys.path.append("/opt/ml/input/project/Utils")
from utils import model_recommend_movies

In [2]:
ml_data_path = "/opt/ml/input/project/Data/EDA/"
data_path = "/opt/ml/input/project/data/"
ratings = pd.read_csv(ml_data_path+"ml-25m/ratings.csv")
movies = pd.read_csv(ml_data_path + "ml-25m/movies.csv")

ratings = ratings.merge(movies[["movieId","title"]],how="left",on = "movieId")                        
ratings.drop(columns = ["timestamp"],inplace = True)
ratings["rating"] = 1

inter_mbti_ml_title_movieid = pd.read_csv(data_path+"230129_Popular_movie_1192_cwj.csv")


In [3]:
user2idx = {k:v for v,k in enumerate(ratings['userId'].unique())}
item2idx = {k:v for v,k in enumerate(ratings['movieId'].unique())}
idx2item = {v:k for v,k in enumerate(ratings['movieId'].unique())}

ratings['useridx'] = ratings['userId'].map(user2idx) # userId -> 인덱스로 바꿔줍니다.
ratings['movieidx'] = ratings['movieId'].map(item2idx) # movieId -> 인덱스로 바꿔줍니다.

In [3]:
inter_mbti_ml_movieid = list(inter_mbti_ml_title_movieid['movieId'].values)
ratings = ratings[ratings['movieId'].isin(inter_mbti_ml_movieid)]

In [38]:
active_user = pd.DataFrame(ratings['userId'].value_counts()>350).reset_index()
active_user.columns = ['userId','active']

In [41]:
ratings = ratings.merge(active_user,how='left',on='userId')
ratings = ratings[ratings['active']==True]
ratings

In [4]:
import random 

user_list = list(ratings['userId'].unique())
test_size = int(len(user_list)*0.1)
random.seed(42)
test_user = random.sample(user_list, test_size)
train_user = list(set(user_list)-set(test_user))

In [5]:
train_df = ratings[ratings['userId'].isin(train_user)]
test_df = ratings[ratings['userId'].isin(test_user)]

In [6]:
print("총 user 수 :",ratings['userId'].nunique())
print("train user 수 : ",len(train_user))
print("test user 수 : ",len(test_user),"\n")
print("총 movieId 수 :",ratings['movieId'].nunique())
print("train movieId 수 : ",train_df["movieId"].nunique())
print("test movieId 수 : ",test_df["movieId"].nunique(),"\n")
print("총 interaction 수 : ", ratings.shape[0])
print("train interaction 수 : ", train_df.shape[0])
print("test interaction 수 : ", test_df.shape[0])

총 user 수 : 161859
train user 수 :  145674
test user 수 :  16185 

총 movieId 수 : 1031
train movieId 수 :  1030
test movieId 수 :  971 

총 interaction 수 :  6348460
train interaction 수 :  5717422
test interaction 수 :  631038


In [7]:
grouped = test_df.groupby('userId')
test_user_movie = grouped['movieId'].apply(list)
test_user_movie_dict = dict(test_user_movie)

In [83]:
grouped = ratings.groupby('userId')
test_user_movie = grouped['movieId'].apply(list)
test_user_movie_dict = dict(test_user_movie)

In [8]:
annoy_model = annoy.AnnoyIndex(64,"angular")
annoy_model.load("./LightGCN_64") # lgcn

True

In [9]:
annoy_model = annoy.AnnoyIndex(64,"angular")
annoy_model.load("./validation_test/ALS_angular_64") # ALS

True

In [9]:
annoy_model = annoy.AnnoyIndex(64,"angular")
annoy_model.load("./validation_test/BPR_64") # BPR

True

In [10]:
from tqdm import tqdm
K = 10
user_scores = {}

for user, movie_list in tqdm(test_user_movie_dict.items()):
    movie_list_len = len(movie_list)
    user_valid_movieids = random.sample(movie_list, min(K,movie_list_len-1)) # 검증할 movieId K개(인터랙션이 K보다 적다면 인터랙션 수-1 만큼만)
    user_prefer_movieids = list(set(movie_list)-set(user_valid_movieids)) # 인터랙션에서 user_valid_movieids를 제외한 영화
    scores = {}

    for mid in user_prefer_movieids: # prefer_movie와 유사도가 높을수록 높은 점수를 부여합니다.
        neighbour, dist = annoy_model.get_nns_by_item(mid,500,include_distances=True)
        norm = 2 # max( sqrt(2-2cos) )
        cnt = 0
        for n,d in zip(neighbour[1:],dist[1:]):
            if cnt == K:
                break
            if n not in user_prefer_movieids: # 유사한 영화가 prefer에 포함되어있으면 안됩니다.
                try:
                    scores[n] += 1-d/norm # angular similarity가 낮을수록 score를 높게 측정합니다.
                except:
                    scores[n] = 0
                    scores[n] += 1-d/norm
                cnt += 1
        assert cnt==K, "갯수가 모자라요"

    sort_scores = sorted(scores.items(),key = lambda x: -x[1])[:K] # score를 내림차순 정렬 해줍니다.
    recall = 0

    for mid,score in sort_scores:
        if mid in user_valid_movieids: # 상위 K개를 뽑았을 때 valid에 포함되어 있으면 +1 해줍니다.
            recall+=1
    user_scores[user] = recall/min(K,movie_list_len-1)

recall10 = sum(list(user_scores.values()))/len(user_scores.keys())

100%|██████████| 16253/16253 [18:57<00:00, 14.29it/s]  


In [54]:
from tqdm import tqdm
K = 10
user_scores = {}
all_user = len(test_user_movie_dict)
hit_user = 0

for user, movie_list in tqdm(test_user_movie_dict.items()):
    movie_list_len = len(movie_list)
    user_valid_movieids = random.sample(movie_list, 1) # 검증할 movieId K개(인터랙션이 K보다 적다면 인터랙션 수-1 만큼만)
    user_prefer_movieids = list(set(movie_list)-set(user_valid_movieids)) # 인터랙션에서 user_valid_movieids를 제외한 영화
    
    scores = {}

    for mid in user_prefer_movieids: # prefer_movie와 유사도가 높을수록 높은 점수를 부여합니다.
        neighbour, dist = annoy_model.get_nns_by_item(mid,500,include_distances=True)
        norm = 2 # max( sqrt(2-2cos) )
        cnt = 0
        for n,d in zip(neighbour[1:],dist[1:]):
            if cnt == K:
                break
            if n not in user_prefer_movieids: # 유사한 영화가 prefer에 포함되어있으면 안됩니다.
                try:
                    scores[n] += 1-d/norm # angular similarity가 낮을수록 score를 높게 측정합니다.
                except:
                    scores[n] = 0
                    scores[n] += 1-d/norm
                cnt += 1
        assert cnt==K, "갯수가 모자라요"

    sort_scores = sorted(scores.items(),key = lambda x: -x[1])[:K] # score를 내림차순 정렬 해줍니다.
    

    for id,_ in sort_scores:
        if user_valid_movieids[0]==id:
            hit_user += 1
            break


Hit10 = hit_user/all_user

 44%|████▍     | 268/611 [03:03<03:54,  1.46it/s]


AssertionError: 갯수가 모자라요

In [9]:
from tqdm import tqdm
K = 10
user_scores = {}
all_user = len(test_user_movie_dict)
hit_user = 0

for user, movie_list in tqdm(test_user_movie_dict.items()):
    movie_list_len = len(movie_list)
    user_valid_movieids = random.sample(movie_list, 1) # 검증할 movieId K개(인터랙션이 K보다 적다면 인터랙션 수-1 만큼만)
    user_prefer_movieids = list(set(movie_list)-set(user_valid_movieids)) # 인터랙션에서 user_valid_movieids를 제외한 영화
    
    if user_valid_movieids[0] in model_recommend_movies(user_prefer_movieids, 10):
        hit_user += 1

    # for mid in user_prefer_movieids: # prefer_movie와 유사도가 높을수록 높은 점수를 부여합니다.
    #     neighbour, dist = annoy_model.get_nns_by_item(mid,500,include_distances=True)
    #     norm = 2 # max( sqrt(2-2cos) )
    #     cnt = 0
    #     for n,d in zip(neighbour[1:],dist[1:]):
    #         if cnt == K:
    #             break
    #         if n not in user_prefer_movieids: # 유사한 영화가 prefer에 포함되어있으면 안됩니다.
    #             try:
    #                 scores[n] += 1-d/norm # angular similarity가 낮을수록 score를 높게 측정합니다.
    #             except:
    #                 scores[n] = 0
    #                 scores[n] += 1-d/norm
    #             cnt += 1
    #     assert cnt==K, "갯수가 모자라요"

    # sort_scores = sorted(scores.items(),key = lambda x: -x[1])[:K] # score를 내림차순 정렬 해줍니다.
    

    # for id,_ in sort_scores:
    #     if user_valid_movieids[0]==id:
    #         hit_user += 1
    #         break


Hit10 = hit_user/all_user

  0%|          | 0/16185 [00:00<?, ?it/s]

/opt/ml/input/project/Utils/../interaction_model/LightGCN_64


  0%|          | 9/16185 [00:00<08:03, 33.49it/s]

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64


  0%|          | 19/16185 [00:00<07:56, 33.92it/s]

...

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64


100%|█████████▉| 16182/16185 [08:59<00:00, 23.09it/s]

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64


100%|██████████| 16185/16185 [08:59<00:00, 29.97it/s]

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10


In [10]:
Hit10

0.13957367933271547

In [26]:
hit_user

4297

In [25]:
recall10 # lgcn, Hit@10임

0.26549274019153535

In [13]:
recall10 # BPR

0.047163820305706115

In [60]:
interactions_df = ratings.sort_values('userId')

In [61]:
# specify the start and end row indices
start_index = 0
end_index = 80

# group the DataFrame by the 'userId' column
grouped = interactions_df.groupby("userId")

# extract the specified rows for each group into a list of DataFrames
result = [group.iloc[start_index:end_index] for _, group in grouped]

# concatenate the list of DataFrames into a single DataFrame
train_df = pd.concat(result)

# specify the start and end row indices
start_index = 80
end_index = 140 #100

# # group the DataFrame by the 'userId' column
# grouped = interactions_df.groupby("userId")

# extract the specified rows for each group into a list of DataFrames
result = [group.iloc[start_index:] for _, group in grouped]

# concatenate the list of DataFrames into a single DataFrame
test_df = pd.concat(result)

In [65]:
cnt = 0
ref = []
rslt = {}
pbar = tqdm(list(zip(train_df.userId, train_df.movieId)))
for i,j in pbar:
    cnt+=1
    ref.append(j)

    if cnt%10 == 0:
        rslt[i] = model_recommend_movies(ref, 10)
        ref = []

  0%|          | 110/48880 [00:00<00:47, 1023.66it/s]

/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interacti

  1%|          | 330/48880 [00:00<00:46, 1050.39it/s]

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../intera

  1%|          | 550/48880 [00:00<00:46, 1050.57it/s]

...

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../intera

100%|█████████▉| 48799/48880 [00:57<00:00, 1126.98it/s]

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../intera

100%|██████████| 48880/48880 [00:57<00:00, 849.73it/s] 

10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10
/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10


In [70]:


a = model_recommend_movies([10,20,30], 10)

/opt/ml/input/project/Utils/../interaction_model/LightGCN_64
10


In [71]:
a

[50, 73, 110, 123, 158, 163, 231, 253, 261, 293]

In [66]:
test_dict = {}
for i,j in zip(test_df.userId, test_df.movieId):
    try:
        test_dict[i].append(j)
    except KeyError:
        test_dict[i] = [j]

In [74]:
len(test_dict)

611

In [76]:
print(i,j)

548 69640


In [1]:
test_dict

NameError: name 'test_dict' is not defined

In [78]:
cnt = 0
for i in test_dict:
    for j in rslt[i]:
        if j in test_dict[i]:
            cnt+=1
            break
print(cnt)

580


In [79]:
print(len(test_df))
print(len(test_dict))
cnt/len(test_dict) # HitRate@10

212415
611


0.9492635024549918